# **File Ingestion and schema validation**

#### **Task**
- Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

- Read the file ( Present approach of reading the file )

- Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

- Perform basic validation on data columns : eg: remove special character , white spaces from the col name

- As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

- Validate number of columns and column name of ingested file with YAML.

- Write the file in pipe separated text file (|) in gz format.

- Create a summary of the file:

- Total number of rows,

- total number of columns

- file size

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


#### **Testing computational efficiency using brute force**

In [ ]:
pip install modin[ray]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 26.4 MB/s 
     |████████████████████████████████| 141 kB 48.5 MB/s 
     |████████████████████████████████| 54.5 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 51.5 MB/s 
     |████████████████████████████████| 4.1 MB 52.8 MB/s 
     |████████████████████████████████| 89 kB 9.1 MB/s 
     |████████████████████████████████| 128 kB 68.2 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 201 kB 71.9 MB/s 
     |████████████████████████████████| 3.1 MB 43.5 MB/s 
     |████████████████████████████████| 281 kB 53.0 MB/s 
     |████████████████████████████████| 58 kB 4.6 MB/s 
     |████████████████████████████████| 466 kB 57.0 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0rc1-py3-none-any.whl size=18872 sha256=91a101781f0d34085755f0342e095c234568e561216032e07027953b2b071a7

In [ ]:
import dask.dataframe as dd
import modin.pandas as mpd
import time
import ray
import pandas as pd

In [ ]:
import time
import resource 

time_start = time.perf_counter()
df = dd.read_table('/content/drive/MyDrive/data.csv') 
time_elapsed = (time.perf_counter() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print ("%5.1f secs %5.1f MByte" % (time_elapsed,memMb))

  0.0 secs   1.9 MByte


In [ ]:
time_start = time.perf_counter()
df1 = mpd.read_csv('/content/drive/MyDrive/data.csv') 
time_elapsed = (time.perf_counter() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print ("%5.1f secs %5.1f MByte" % (time_elapsed,memMb))

 21.0 secs   1.9 MByte


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [ ]:
time_start = time.perf_counter()
df3 = pd.read_csv('/content/drive/MyDrive/data.csv') 
time_elapsed = (time.perf_counter() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print ("%5.1f secs %5.1f MByte" % (time_elapsed,memMb))

 18.9 secs   1.6 MByte


In [ ]:
df3.head()

,Unnamed: 0,id,cell_id,source,ancestor_id,pct_rank
0,1,0944b58318b789,fced0b7a,# Spectrogram-based CNN for the Tensorflow Spe...,0212b702,0.0
1,3,59958672e3bf59,9b68df7e,# A simple explanation and implementation of D...,8f0f9cda,0.0
2,4,b22e24942614c9,fa089df9,Name: Aviral Jain | \nRoll No: 18AG3AI08 | \nB...,52b2390a,0.0
3,5,59959edc72f7c1,815f8d1f,"<img src=""https://drive.google.com/uc?export=d...",2ba4272a,0.0
4,7,599548cea78ff7,7b084d0a,# DATA UNDESRTANDING\n\n#### Churn veri seti i...,ce055a99,0.0


#### **Removing white spaces or special characters**

In [ ]:
#Replacing white spaces from pandas datafreme 
df3.columns = df3.columns.str.replace(' ','_')
#Replacing special characters in a column with white space
#df['B'].str.replace('[^\w\s]', '')

In [ ]:
df3.head()

,Unnamed:_0,id,cell_id,source,ancestor_id,pct_rank
0,1,0944b58318b789,fced0b7a,# Spectrogram-based CNN for the Tensorflow Spe...,0212b702,0.0
1,3,59958672e3bf59,9b68df7e,# A simple explanation and implementation of D...,8f0f9cda,0.0
2,4,b22e24942614c9,fa089df9,Name: Aviral Jain | \nRoll No: 18AG3AI08 | \nB...,52b2390a,0.0
3,5,59959edc72f7c1,815f8d1f,"<img src=""https://drive.google.com/uc?export=d...",2ba4272a,0.0
4,7,599548cea78ff7,7b084d0a,# DATA UNDESRTANDING\n\n#### Churn veri seti i...,ce055a99,0.0


### **Writing YAML file**

In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: penguins
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - CulmenLength
    - CulmenDepth
    - FlipperLength
    - BodyMass
    - Species

Overwriting file.yaml


In [ ]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")


In [ ]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'penguins',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['CulmenLength',
  'CulmenDepth',
  'FlipperLength',
  'BodyMass',
  'Species']}

In [ ]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("penguins.csv",delimiter=',')
df_sample.head()

,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
0,39.1,18.7,181.0,3750.0,0
1,39.5,17.4,186.0,3800.0,0
2,40.3,18.0,195.0,3250.0,0
3,NaN,NaN,NaN,NaN,0
4,36.7,19.3,193.0,3450.0,0


In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
0,39.1,18.7,181.0,3750.0,0
1,39.5,17.4,186.0,3800.0,0
2,40.3,18.0,195.0,3250.0,0
3,NaN,NaN,NaN,NaN,0
4,36.7,19.3,193.0,3450.0,0


In [ ]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['culmenlength', 'culmendepth', 'flipperlength', 'bodymass', 'species'], dtype='object')
columns of YAML are: ['CulmenLength', 'CulmenDepth', 'FlipperLength', 'BodyMass', 'Species']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [ ]:
# Create csv with gzip compression
df.to_csv('gzfile.csv.gz',
      sep='|',
      header=True,
      index=False,
      compression='gzip')

### **Summary of file**

In [3]:
import os

gz_file = pd.read_csv('gzfile.csv.gz', compression='gzip', sep='|')

data = {
    'Total_number_of_rows' : gz_file.shape[0],
    'Total_number_of_columns' : gz_file.shape[1],
    'file_size' : os.path.getsize("gzfile.csv.gz")}
summarydata = pd.DataFrame(data, columns= ['Total_number_of_rows', 
                                           'Total_number_of_columns',
                                           'file_size'], index=[0])
summarydata

,Total_number_of_rows,Total_number_of_columns,file_size
0,344,5,2384
